# **Preliminary Analysis**

In [3]:
# Necessary modules
import pandas as pd# for data manipulation and analysis
import numpy as np # for numerical computations
from scipy import stats#for statistical functions and hypothesis testing
from mlxtend.preprocessing import minmax_scaling # for feature sclaing(normalization)
import seaborn as sns# for data visualization 
import matplotlib.pyplot as plt#for plotting ad visualizations
import datetime # for working with dates and times
import charset_normalizer# for detecting and normalizing character encoding in text data
import fuzzywuzzy# fir fuzzy string matching
from fuzzywuzzy import process# for extracting best fuzzy matches
#set seed reproducibility(ensures results are consister each time the code is run
np.random.seed(0)

In [5]:
adm_data=pd.read_csv("HDHI Admission data.csv")
adm_data

,SNO,MRD No.,D.O.A,D.O.D,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1,234735,4/1/2017,4/3/2017,81,M,R,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
1,2,234696,4/1/2017,4/5/2017,65,M,R,E,Apr-17,5,...,0,0,0,0,0,0,0,0,0,0
2,3,234882,4/1/2017,4/3/2017,53,M,U,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
3,4,234635,4/1/2017,4/8/2017,67,F,U,E,Apr-17,8,...,0,0,0,0,0,0,0,0,0,0
4,5,234486,4/1/2017,4/23/2017,60,F,U,E,Apr-17,23,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15752,15753,699585,31/03/2019,04/04/2019,86,F,U,O,Mar-19,5,...,0,0,0,0,0,0,0,0,0,0
15753,15754,699500,3/31/2019,4/1/2019,50,M,R,E,Mar-19,2,...,0,0,0,0,0,0,0,0,0,0
15754,15755,700415,31/03/2019,09/04/2019,82,M,U,E,Mar-19,10,...,0,0,0,0,0,0,0,0,0,0
15755,15756,699524,31/03/2019,03/04/2019,59,F,U,O,Mar-19,4,...,0,0,0,0,0,0,0,0,0,0


## How does the data look like?

In [18]:
row_columns=adm_data.shape
datatypes=adm_data.dtypes
print("The number of rows and columns are:" , row_columns)
print(datatypes)

The number of rows and columns are: (15757, 56)
SNO                                 int64
MRD No.                            object
D.O.A                              object
D.O.D                              object
AGE                                 int64
GENDER                             object
RURAL                              object
TYPE OF ADMISSION-EMERGENCY/OPD    object
month year                         object
DURATION OF STAY                    int64
duration of intensive unit stay     int64
OUTCOME                            object
SMOKING                             int64
ALCOHOL                             int64
DM                                  int64
HTN                                 int64
CAD                                 int64
PRIOR CMP                           int64
CKD                                 int64
HB                                 object
TLC                                object
PLATELETS                          object
GLUCOSE                     

## ***Handling missing values***

In [48]:
adm_data.head()

,SNO,MRD No.,D.O.A,D.O.D,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1,234735,4/1/2017,4/3/2017,81,M,R,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
1,2,234696,4/1/2017,4/5/2017,65,M,R,E,Apr-17,5,...,0,0,0,0,0,0,0,0,0,0
2,3,234882,4/1/2017,4/3/2017,53,M,U,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
3,4,234635,4/1/2017,4/8/2017,67,F,U,E,Apr-17,8,...,0,0,0,0,0,0,0,0,0,0
4,5,234486,4/1/2017,4/23/2017,60,F,U,E,Apr-17,23,...,0,0,0,0,0,0,0,0,0,0


### *How many missing **data points** do we have?*

In [24]:
# getting the number of missing data points per column
missing_values_count=adm_data.isnull().sum()
# look at the number of missing points in each column
missing_values_count


SNO                                   0
MRD No.                               0
D.O.A                                 0
D.O.D                                 0
AGE                                   0
GENDER                                0
RURAL                                 0
TYPE OF ADMISSION-EMERGENCY/OPD       0
month year                            0
DURATION OF STAY                      0
duration of intensive unit stay       0
OUTCOME                               0
SMOKING                               0
ALCOHOL                               0
DM                                    0
HTN                                   0
CAD                                   0
PRIOR CMP                             0
CKD                                   0
HB                                  252
TLC                                 286
PLATELETS                           285
GLUCOSE                             863
UREA                                241
CREATININE                          247


In [26]:
missing_values_count[19:28]# for only the features(columns) with missing data

HB                         252
TLC                        286
PLATELETS                  285
GLUCOSE                    863
UREA                       241
CREATININE                 247
BNP                       8441
RAISED CARDIAC ENZYMES       0
EF                        1505
dtype: int64

In [30]:
# How many total missing vaues do we have?
total_cells=np.product(adm_data.shape)
total_missing=missing_values_count.sum()
# percentage of data that's missing
percent_missing=(total_missing/total_cells)*100
percent_missing

1.3735391979981686

## *Why is the data missing?*

In [35]:
missing_values_count[19:28]

HB                         252
TLC                        286
PLATELETS                  285
GLUCOSE                    863
UREA                       241
CREATININE                 247
BNP                       8441
RAISED CARDIAC ENZYMES       0
EF                        1505
dtype: int64

In [50]:
# lets view the first fews rows of the columns with missing values to decide how we will handle them
adm_data.loc[:,'HB':'EF'].head()

,HB,TLC,PLATELETS,GLUCOSE,UREA,CREATININE,BNP,RAISED CARDIAC ENZYMES,EF
0,9.5,16.1,337,80,34,0.9,1880,1,35
1,13.7,9,149,112,18,0.9,NaN,0,42
2,10.6,14.7,329,187,93,2.3,210,0,NaN
3,12.8,9.9,286,130,27,0.6,NaN,0,42
4,13.6,9.1,26,144,55,1.25,1840,0,16


In [54]:
subset_adm_data=adm_data.loc[:,'HB':'EF'].head()
subset_adm_data.fillna(0)

,HB,TLC,PLATELETS,GLUCOSE,UREA,CREATININE,BNP,RAISED CARDIAC ENZYMES,EF
0,9.5,16.1,337,80,34,0.9,1880,1,35
1,13.7,9,149,112,18,0.9,0,0,42
2,10.6,14.7,329,187,93,2.3,210,0,0
3,12.8,9.9,286,130,27,0.6,0,0,42
4,13.6,9.1,26,144,55,1.25,1840,0,16


In [56]:
adm_data.loc[:,'HB':'EF']=adm_data.loc[:,'HB':'EF'].fillna(0)

In [58]:
adm_data.isnull().sum()

SNO                                0
MRD No.                            0
D.O.A                              0
D.O.D                              0
AGE                                0
GENDER                             0
RURAL                              0
TYPE OF ADMISSION-EMERGENCY/OPD    0
month year                         0
DURATION OF STAY                   0
duration of intensive unit stay    0
OUTCOME                            0
SMOKING                            0
ALCOHOL                            0
DM                                 0
HTN                                0
CAD                                0
PRIOR CMP                          0
CKD                                0
HB                                 0
TLC                                0
PLATELETS                          0
GLUCOSE                            0
UREA                               0
CREATININE                         0
BNP                                0
RAISED CARDIAC ENZYMES             0
E

In [68]:
adm_data.columns

Index(['SNO', 'MRD No.', 'D.O.A', 'D.O.D', 'AGE', 'GENDER', 'RURAL',
       'TYPE OF ADMISSION-EMERGENCY/OPD', 'month year', 'DURATION OF STAY',
       'duration of intensive unit stay', 'OUTCOME', 'SMOKING ', 'ALCOHOL',
       'DM', 'HTN', 'CAD', 'PRIOR CMP', 'CKD', 'HB', 'TLC', 'PLATELETS',
       'GLUCOSE', 'UREA', 'CREATININE', 'BNP', 'RAISED CARDIAC ENZYMES', 'EF',
       'SEVERE ANAEMIA', 'ANAEMIA', 'STABLE ANGINA', 'ACS', 'STEMI',
       'ATYPICAL CHEST PAIN', 'HEART FAILURE', 'HFREF', 'HFNEF', 'VALVULAR',
       'CHB', 'SSS', 'AKI', 'CVA INFRACT', 'CVA BLEED', 'AF', 'VT', 'PSVT',
       'CONGENITAL', 'UTI', 'NEURO CARDIOGENIC SYNCOPE', 'ORTHOSTATIC',
       'INFECTIVE ENDOCARDITIS', 'DVT', 'CARDIOGENIC SHOCK', 'SHOCK',
       'PULMONARY EMBOLISM', 'CHEST INFECTION'],
      dtype='object')

## Scaling and normalizing our data

In [85]:
adm_data.dtypes

SNO                                 int64
MRD No.                            object
D.O.A                              object
D.O.D                              object
AGE                                 int64
GENDER                             object
RURAL                              object
TYPE OF ADMISSION-EMERGENCY/OPD    object
month year                         object
DURATION OF STAY                    int64
duration of intensive unit stay     int64
OUTCOME                            object
SMOKING                             int64
ALCOHOL                             int64
DM                                  int64
HTN                                 int64
CAD                                 int64
PRIOR CMP                           int64
CKD                                 int64
HB                                 object
TLC                                object
PLATELETS                          object
GLUCOSE                            object
UREA                              

In [77]:
#selecting only numeric columns
num_cols=adm_data.select_dtypes(include=['int64','float64']).columns.tolist()
print("Numeric columns:",num_cols)

Numeric columns: ['SNO', 'AGE', 'DURATION OF STAY', 'duration of intensive unit stay', 'SMOKING ', 'ALCOHOL', 'DM', 'HTN', 'CAD', 'PRIOR CMP', 'CKD', 'RAISED CARDIAC ENZYMES', 'SEVERE ANAEMIA', 'ANAEMIA', 'STABLE ANGINA', 'ACS', 'STEMI', 'ATYPICAL CHEST PAIN', 'HEART FAILURE', 'HFREF', 'HFNEF', 'VALVULAR', 'CHB', 'SSS', 'AKI', 'CVA INFRACT', 'CVA BLEED', 'AF', 'VT', 'PSVT', 'CONGENITAL', 'UTI', 'NEURO CARDIOGENIC SYNCOPE', 'ORTHOSTATIC', 'INFECTIVE ENDOCARDITIS', 'DVT', 'CARDIOGENIC SHOCK', 'SHOCK', 'PULMONARY EMBOLISM']


In [79]:
# detecting categorical columns
cat_cols=adm_data.select_dtypes(include=['object','category']).columns.tolist()
print('Categorical columns:',cat_cols)

Categorical columns: ['MRD No.', 'D.O.A', 'D.O.D', 'GENDER', 'RURAL', 'TYPE OF ADMISSION-EMERGENCY/OPD', 'month year', 'OUTCOME', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'BNP', 'EF', 'CHEST INFECTION']


In [83]:
#Handle binary categorical featutes
# move binary cols to categorical
binary_cols=[col for col in num_cols if adm_data[col].dropna().value_counts().index.isin([0,1]).all()]
#update lists
num_cols=[col for col in num_cols if col not in binary_cols]
cat_cols=cat_cols + binary_cols
print("Final numeric:",num_cols)
print("Final categorical:",cat_cols)

Final numeric: ['SNO', 'AGE', 'DURATION OF STAY', 'duration of intensive unit stay']
Final categorical: ['MRD No.', 'D.O.A', 'D.O.D', 'GENDER', 'RURAL', 'TYPE OF ADMISSION-EMERGENCY/OPD', 'month year', 'OUTCOME', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'BNP', 'EF', 'CHEST INFECTION', 'SMOKING ', 'ALCOHOL', 'DM', 'HTN', 'CAD', 'PRIOR CMP', 'CKD', 'RAISED CARDIAC ENZYMES', 'SEVERE ANAEMIA', 'ANAEMIA', 'STABLE ANGINA', 'ACS', 'STEMI', 'ATYPICAL CHEST PAIN', 'HEART FAILURE', 'HFREF', 'HFNEF', 'VALVULAR', 'CHB', 'SSS', 'AKI', 'CVA INFRACT', 'CVA BLEED', 'AF', 'VT', 'PSVT', 'CONGENITAL', 'UTI', 'NEURO CARDIOGENIC SYNCOPE', 'ORTHOSTATIC', 'INFECTIVE ENDOCARDITIS', 'DVT', 'CARDIOGENIC SHOCK', 'SHOCK', 'PULMONARY EMBOLISM']


In [95]:
from sklearn.preprocessing import MinMaxScaler
num_cols=['SNO', 'AGE', 'DURATION OF STAY', 'duration of intensive unit stay', 'SMOKING ', 'ALCOHOL', 'DM', 'HTN', 'CAD', 'PRIOR CMP', 'CKD', 'RAISED CARDIAC ENZYMES', 'SEVERE ANAEMIA', 'ANAEMIA', 'STABLE ANGINA', 'ACS', 'STEMI', 'ATYPICAL CHEST PAIN', 'HEART FAILURE', 'HFREF', 'HFNEF', 'VALVULAR', 'CHB', 'SSS', 'AKI', 'CVA INFRACT', 'CVA BLEED', 'AF', 'VT', 'PSVT', 'CONGENITAL', 'UTI', 'NEURO CARDIOGENIC SYNCOPE', 'ORTHOSTATIC', 'INFECTIVE ENDOCARDITIS', 'DVT', 'CARDIOGENIC SHOCK', 'SHOCK', 'PULMONARY EMBOLISM']
scaler=MinMaxScaler()
adm_data[num_cols]=scaler.fit_transform(adm_data[num_cols])
adm_data[num_cols].head()

,SNO,AGE,DURATION OF STAY,duration of intensive unit stay,SMOKING,ALCOHOL,DM,HTN,CAD,PRIOR CMP,...,PSVT,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM
0,0.000000,0.726415,0.020619,0.034483,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000063,0.575472,0.041237,0.034483,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000127,0.462264,0.020619,0.051724,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000190,0.594340,0.072165,0.103448,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000254,0.528302,0.226804,0.155172,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
adm_data.to_csv("adm_data.csv")